# Polynomial Regression and Overfitting

In this module, you will learn how to work with polynomial regression in scikit-learn. We will also introduce the concept of overfitting and learn how to avoid this.

<b>Functions and attributes in this lecture: </b>
- `pandas:` - Pandas package with alias `pd`
 - `.copy()` - Make a copy of a pandas Dataframe
- `sklearn.preprocessing` - Submodule for preprocessing tools like PolynomialFeatures
  - `PolynomialFeatures` - For creating of polynomial features
  - `.fit_transform()` - Fitting and transforming the data to form new polynomial features
- `sklearn.metrics` - Submodule for metrics used to evaluate models
  - `mean_absolute_error` - Taking the mean-abosolute-error of a vector

In [1]:
# Non-sklearn packages
import numpy as np
import pandas as pd

# Sklearn packages
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [2]:
# Load in the diabetes dataset
X, y = load_diabetes(return_X_y=True, as_frame=True)

# Get the description of the dataset
print(load_diabetes()["DESCR"])

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

## Manually Create Polynomial Features

We begin by manually creating features in Pandas and measure how this affects the error rate of our models.

In [3]:
# Check the correlation with the target variable
pd.concat([X,  y], axis=1).corr()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
age,1.000000,0.173737,0.185085,0.335427,0.260061,0.219243,-0.075181,0.203841,0.270777,0.301731,0.187889
sex,0.173737,1.000000,0.088161,0.241013,0.035277,0.142637,-0.379090,0.332115,0.149918,0.208133,0.043062
bmi,0.185085,0.088161,1.000000,0.395415,0.249777,0.261170,-0.366811,0.413807,0.446159,0.388680,0.586450
bp,0.335427,0.241013,0.395415,1.000000,0.242470,0.185558,-0.178761,0.257653,0.393478,0.390429,0.441484
s1,0.260061,0.035277,0.249777,0.242470,1.000000,0.896663,0.051519,0.542207,0.515501,0.325717,0.212022
s2,0.219243,0.142637,0.261170,0.185558,0.896663,1.000000,-0.196455,0.659817,0.318353,0.290600,0.174054
s3,-0.075181,-0.379090,-0.366811,-0.178761,0.051519,-0.196455,1.000000,-0.738493,-0.398577,-0.273697,-0.394789
s4,0.203841,0.332115,0.413807,0.257653,0.542207,0.659817,-0.738493,1.000000,0.617857,0.417212,0.430453
s5,0.270777,0.149918,0.446159,0.393478,0.515501,0.318353,-0.398577,0.617857,1.000000,0.464670,0.565883
s6,0.301731,0.208133,0.388680,0.390429,0.325717,0.290600,-0.273697,0.417212,0.464670,1.000000,0.382483


In [4]:
# Selecting only some of the columns
X_selected = X[["bmi", "bp", "s4", "s5"]]

In [5]:
# Creating a copy for further modification
X_cross_terms = X_selected.copy()

In [6]:
# Manually creating cross_terms
X_cross_terms["bmi_times_bp"] = X_cross_terms["bmi"] * X_cross_terms["bp"]
X_cross_terms["bp_times_s5"] = X_cross_terms["bp"] * X_cross_terms["s5"]

In [7]:
# Shows the new columns we have created
X_cross_terms.head()

,bmi,bp,s4,s5,bmi_times_bp,bp_times_s5
0,0.061696,0.021872,-0.002592,0.019908,0.001349,0.000435
1,-0.051474,-0.026328,-0.039493,-0.068330,0.001355,0.001799
2,0.044451,-0.005671,-0.002592,0.002864,-0.000252,-0.000016
3,-0.011595,-0.036656,0.034309,0.022692,0.000425,-0.000832
4,-0.036385,0.021872,-0.002592,-0.031991,-0.000796,-0.000700


In [8]:
# Showing the correlation again
pd.concat([X_cross_terms,  y], axis=1).corr()

,bmi,bp,s4,s5,bmi_times_bp,bp_times_s5,target
bmi,1.000000,0.395415,0.413807,0.446159,0.059822,-0.002849,0.586450
bp,0.395415,1.000000,0.257653,0.393478,0.117993,0.099788,0.441484
s4,0.413807,0.257653,1.000000,0.617857,-0.087453,0.026741,0.430453
s5,0.446159,0.393478,0.617857,1.000000,-0.002515,0.083316,0.565883
bmi_times_bp,0.059822,0.117993,-0.087453,-0.002515,1.000000,0.410842,0.147113
bp_times_s5,-0.002849,0.099788,0.026741,0.083316,0.410842,1.000000,0.099435
target,0.586450,0.441484,0.430453,0.565883,0.147113,0.099435,1.000000


In [9]:
# Splitting into test sets and training sets
X_train, X_test, y_train, y_test = train_test_split(X_cross_terms, y, test_size=0.33, random_state=42)

In [10]:
# Creating, training, and predicting the polynomial model
poly_reg = LinearRegression()
poly_reg.fit(X_train, y_train)
y_pred_poly = poly_reg.predict(X_test)

In [11]:
# Use mean-absolute-error to measure the model
from sklearn.metrics import mean_absolute_error, mean_squared_error
print("Poly Error:", mean_absolute_error(y_pred_poly, y_test))

Poly Error: 42.93438227273901


In [12]:
# Create a linear model
lin_reg = LinearRegression()
lin_reg.fit(X_train[["bmi", "bp", "s4", "s5"]], y_train)
y_pred_lin = lin_reg.predict(X_test[["bmi", "bp", "s4", "s5"]])

In [13]:
# Shows score of the purely linear model
print("Linear Error:", mean_absolute_error(y_pred_lin, y_test))

Linear Error: 43.56264989352488


## Using Scikit-Learns Polynomial Features

We now introduce scikit-learn's built in PolynomialFeatures for handling polynomial featues.

In [14]:
# Importing polynomial features
from sklearn.preprocessing import PolynomialFeatures

In [15]:
# We are working with four features
X_selected.head()

,bmi,bp,s4,s5
0,0.061696,0.021872,-0.002592,0.019908
1,-0.051474,-0.026328,-0.039493,-0.068330
2,0.044451,-0.005671,-0.002592,0.002864
3,-0.011595,-0.036656,0.034309,0.022692
4,-0.036385,0.021872,-0.002592,-0.031991


In [16]:
# Creates polynomial features automatically
polynomial_features = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
X_poly = polynomial_features.fit_transform(X_selected)

In [17]:
# Split into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.33, random_state=42)

In [18]:
# Create model, train, and test
poly_reg = LinearRegression()
poly_reg.fit(X_train, y_train)
y_pred_poly = poly_reg.predict(X_test)

In [19]:
# Getting the mean-absolute-error score
print("Poly Error:", mean_absolute_error(y_pred_poly, y_test))

Poly Error: 43.19173169395458


## Fitting Everything Into a Pipeline

It is time to fit the previous steps into a pipeline for better reproducibility and to avoid data leakage.

In [20]:
# We are working with four features
X_selected.head()

,bmi,bp,s4,s5
0,0.061696,0.021872,-0.002592,0.019908
1,-0.051474,-0.026328,-0.039493,-0.068330
2,0.044451,-0.005671,-0.002592,0.002864
3,-0.011595,-0.036656,0.034309,0.022692
4,-0.036385,0.021872,-0.002592,-0.031991


In [21]:
# Dividing into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.33, random_state=42)

In [22]:
# Creating a pipeline (use scaler after the polynomial features!)
pipe = Pipeline([
    ("poly_features", PolynomialFeatures(degree=2)),
    ("scaler", StandardScaler()),
    ("linear_regression", LinearRegression())
])

In [23]:
# Fitting and predicting
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

In [24]:
# We get the same score
print("Poly error:", mean_absolute_error(y_pred, y_test))

Poly error: 43.92474058356314


## Checking if you are Overfitting

We will now loop through several polynomial degrees to check which one is best.

In [25]:
# We are working with four features
X_selected.head()

,bmi,bp,s4,s5
0,0.061696,0.021872,-0.002592,0.019908
1,-0.051474,-0.026328,-0.039493,-0.068330
2,0.044451,-0.005671,-0.002592,0.002864
3,-0.011595,-0.036656,0.034309,0.022692
4,-0.036385,0.021872,-0.002592,-0.031991


In [26]:
# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.33, random_state=42)

In [27]:
# Checking the error for various degrees
for i in range(1, 6):
    pipe = Pipeline([
        ("poly_features", PolynomialFeatures(degree=i)),
        ("scaler", StandardScaler()),
        ("linear_regression", LinearRegression())
    ])
    
    pipe.fit(X_train, y_train)
    y_pred_train = pipe.predict(X_train)
    y_pred_test = pipe.predict(X_test)
    
    print(
        f"| Degree {i} on train:", 
        round(mean_squared_error(y_pred_train, y_train), 2), 
        f"| Degree {i} on test:", 
        round(mean_squared_error(y_pred_test, y_test), 2),
        " |"
    )

| Degree 1 on train: 3209.99 | Degree 1 on test: 2906.08  |
| Degree 2 on train: 3076.24 | Degree 2 on test: 2903.13  |
| Degree 3 on train: 2918.27 | Degree 3 on test: 3034.71  |
| Degree 4 on train: 2538.65 | Degree 4 on test: 4070.83  |
| Degree 5 on train: 1950.11 | Degree 5 on test: 23934.96  |
